<a href="https://colab.research.google.com/github/agu3ro/PyTorch-DL-Linkedin-Learning/blob/main/05_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [12]:
# Model development

class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=torch.nn.Conv2d(3,6,3,1)
    self.conv2=torch.nn.Conv2d(6,16,3,1)
    self.fc1 = torch.nn.Linear(16*6*6,128)
    self.fc2 = torch.nn.Linear(128,84)
    self.fc3 = torch.nn.Linear(84,10)

  def forward(self,x):
    x = torch.nn.functional.relu(self.conv1(x))
    x = torch.nn.functional.max_pool2d(x,2,2)
    x = torch.nn.functional.relu(self.conv2(x))
    x = torch.nn.functional.max_pool2d(x,2,2)
    x = x.view(-1,16*6*6)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [13]:
# Instantiate the Model


model = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [14]:
# Load and transform the data


train_data = datasets.CIFAR10(root='./data', download=True, train=True,
                              transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]))
test_data =  datasets.CIFAR10(root='./data', download=True, train=False,
                              transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]))
train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Train the Network

for i in range(10):
  running_loss = 0 # mean loss thru 2k iterations
  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
    if b%2000==1999:
      print(f"Epoch: {i}, {b+1}, loss: {loss}, mean loss after 2k iterations: {running_loss/2000}")
      running_loss = 0.0 # 0.0 cause float

Epoch: 0, 2000, loss: 1.8645915985107422, mean loss after 2k iterations: 2.2103153473734856
Epoch: 0, 4000, loss: 1.7792268991470337, mean loss after 2k iterations: 2.1039475839734076
Epoch: 0, 6000, loss: 1.7690708637237549, mean loss after 2k iterations: 2.055492562055588
Epoch: 0, 8000, loss: 1.8195528984069824, mean loss after 2k iterations: 1.9897192055583
Epoch: 0, 10000, loss: 2.058522939682007, mean loss after 2k iterations: 1.9166234122514725
Epoch: 0, 12000, loss: 1.4310052394866943, mean loss after 2k iterations: 1.8182882807850838
Epoch: 1, 2000, loss: 1.6005637645721436, mean loss after 2k iterations: 1.7292324734330178
Epoch: 1, 4000, loss: 1.7877137660980225, mean loss after 2k iterations: 1.6934224369972943
Epoch: 1, 6000, loss: 1.7478904724121094, mean loss after 2k iterations: 1.626327946484089
Epoch: 1, 8000, loss: 1.4219653606414795, mean loss after 2k iterations: 1.5890781240463256
Epoch: 1, 10000, loss: 1.206368088722229, mean loss after 2k iterations: 1.554744760